Will Albert   
DSSA 5102  
Mini 2: Web Scraping to SQL  

### Survivor - Individual Challenge Wins by Season

In [1]:
import requests
import pandas as pd
import sqlite3
import lxml.html

In [2]:
x = "https://www.truedorktimes.com/survivor/boxscores/indchw-season.htm"
r = requests.get(x)

In [3]:
html = lxml.html.fromstring(r.text)
wins = html.get_element_by_id("boxscore")
tablebody = wins.getchildren()[2]
tablehead = wins.getchildren()[1]

In [4]:
data = {
    "Rank": [],
    "Contestant": [],
    "Season": [],
    "Individual_challenge_wins": [],
    "Individual_challenge_appearances": []
}

for row in tablebody.getchildren():
    data["Rank"].append(row.getchildren()[0].text_content())
    data["Contestant"].append(row.getchildren()[1].text_content().strip())
    data["Season"].append(row.getchildren()[2].text_content())
    data["Individual_challenge_wins"].append(row.getchildren()[3].text_content().strip("*"))
    data["Individual_challenge_appearances"].append(row.getchildren()[4].text_content().strip("*"))
    
df = pd.DataFrame(data)

In [5]:
df.Individual_challenge_wins = df.Individual_challenge_wins.astype(int)
df.Individual_challenge_appearances = df.Individual_challenge_appearances.astype(int)

In [6]:
db = sqlite3.connect(":memory:")
cursor = db.cursor()
cursor.execute("""
    CREATE TABLE WINS(RANK, CONTESTANT, SEASON, IND_WINS, IND_APPEARANCES)
""")

for row in df.itertuples():
    insert_sql_syntax = """
    INSERT INTO WINS(RANK, CONTESTANT, SEASON, IND_WINS, IND_APPEARANCES)
    VALUES (?,?,?,?,?)
    """
    cursor.execute(insert_sql_syntax, row[1:])
#db.close()

In [7]:
for row in cursor.execute("""
    SELECT * 
    FROM WINS
    WHERE IND_APPEARANCES = 11
"""):
    print(row)

('1', 'Colby Donaldson', 'S2 - The Australian Outback', 7, 11)
('1', 'Terry Deitz', 'S12 - Panama', 7, 11)
('4', 'Mike Holloway', 'S30 - Worlds Apart', 6, 11)
('10', 'Chrissy Hofbeck', 'S35 - Heroes v Healers v Hustlers', 5, 11)
('10', 'Lex van den Berghe', 'S3 - Africa', 5, 11)
('20', 'Malcolm Freberg', 'S25 - Philippines', 4, 11)
('20', 'Michele Fitzgerald', 'S32 - Kaoh Rong', 4, 11)
('20', 'Ken McNickle', 'S33 - Millennials vs. Gen X', 4, 11)
('43', 'Eliza Orlins', 'S9 - Vanuatu', 3, 11)
('43', 'Amanda Kimmel', 'S15 - China', 3, 11)
('43', 'Mike Skupin', 'S25 - Philippines', 3, 11)


In [8]:
for row in cursor.execute("""
    SELECT CONTESTANT, SEASON, IND_WINS 
    FROM WINS
    WHERE IND_APPEARANCES = 11
"""):
    print(row)

('Colby Donaldson', 'S2 - The Australian Outback', 7)
('Terry Deitz', 'S12 - Panama', 7)
('Mike Holloway', 'S30 - Worlds Apart', 6)
('Chrissy Hofbeck', 'S35 - Heroes v Healers v Hustlers', 5)
('Lex van den Berghe', 'S3 - Africa', 5)
('Malcolm Freberg', 'S25 - Philippines', 4)
('Michele Fitzgerald', 'S32 - Kaoh Rong', 4)
('Ken McNickle', 'S33 - Millennials vs. Gen X', 4)
('Eliza Orlins', 'S9 - Vanuatu', 3)
('Amanda Kimmel', 'S15 - China', 3)
('Mike Skupin', 'S25 - Philippines', 3)
